In [1]:
#@title Check GPU Status
import subprocess
simple_nvidia_smi_display = True#@param {type:"boolean"}
if simple_nvidia_smi_display:
    #!nvidia-smi
    nvidiasmi_output = subprocess.run(['nvidia-smi', '-L'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
else:
    #!nvidia-smi -i 0 -e 0
    nvidiasmi_output = subprocess.run(['nvidia-smi'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_output)
    nvidiasmi_ecc_note = subprocess.run(['nvidia-smi', '-i', '0', '-e', '0'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(nvidiasmi_ecc_note)

GPU 0: Tesla T4 (UUID: GPU-64c0a9f6-6d75-bfe4-7430-10e57b07bbec)



In [ ]:
#@title Prepare folders
#@markdown **UPLOAD YOUR TRAINING DATASET TO GOOGLE DRIVE**
import subprocess, os, sys, ipykernel

def gitclone(url, targetdir=None):
    if targetdir:
        res = subprocess.run(['git', 'clone', url, targetdir], stdout=subprocess.PIPE).stdout.decode('utf-8')
    else:
        res = subprocess.run(['git', 'clone', url], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipi(modulestr):
    res = subprocess.run(['pip', 'install', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def pipie(modulestr):
    res = subprocess.run(['git', 'install', '-e', modulestr], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

def wget(url, outputdir):
    res = subprocess.run(['wget', url, '-P', f'{outputdir}'], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(res)

try:
    from google.colab import drive
    print("Google Colab detected. Using Google Drive.")
    is_colab = True
    google_drive = True #@param {type:"boolean"}
#@markdown CHECK TO LOAD/SAVE FILES TO GOOGLE DRIVE
    save_models_to_google_drive = True #@param {type:"boolean"}
except:
    is_colab = False
    google_drive = False
    save_models_to_google_drive = False
    print("Google Colab not detected.")

if is_colab:
    if google_drive is True:
        drive.mount('/content/drive')
        root_path = '/content/drive/MyDrive/AI/KICK_GNR8R/SPU'
    else:
        root_path = '/content'
else:
    root_path = os.getcwd()

import os
def createPath(filepath):
    os.makedirs(filepath, exist_ok=True)

initDirPath = f'{root_path}/init_audio'
createPath(initDirPath)
outDirPath = f'{root_path}/audio_out'
createPath(outDirPath)

if is_colab:
    if google_drive and not save_models_to_google_drive or not google_drive:
        model_path = '/content/SPU'
        createPath(model_path)
    if google_drive and save_models_to_google_drive:
        model_path = f'{root_path}/models'
        createPath(model_path)
else:
    model_path = f'{root_path}/models'
    createPath(model_path)

#@markdown CHECK TO SAVE MODELS TO WEIGHTS AND BIASES
save_models_to_wandb = True #@param {type:"boolean"}
if save_models_to_wandb == True:
    save_wandb = 'all'
    print('Saving model checkpoints in wandb')

In [5]:
#@title INSTALL DEPENDENCIES
#@markdown Restart runtaime from code output prompt, then run "Install Dependencies" again to complete installation.
!git clone https://github.com/NXTDMNSN/KICK-GNR8R-SPU.git
!pip install /content/KICK-GNR8R-SPU
!pip install -r /content/KICK-GNR8R-SPU/requirements.txt


fatal: destination path 'KICK-GNR8R-SPU' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./KICK-GNR8R-SPU
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-audiomentations: filename=torch_audiomentations-0.11.0-py3-none-any.whl size=48220 sha256=ba2d5b5ab9ccc17f388283ace459ef065a7840ddba30bac97797a3b3d8040dd4
  Stored in directory: /root/.cache/pip/wheels/52/27/c5/51790e0772c984a1c75e81dc42f11fc8b11db810601850ae33
Successfully built torch-audiomentations
  Attempting uninstall: torch-audiomentations
    Found existing installation: torch-audiomentations 0.11.0
    Uninstalling torch-audiomentations-0.11.0:
      Successfully uninstalled torch-audiomentations-0.11.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: C

# V1


In [ ]:
import argparse
import piano_tuner
import pymir
import librosa

# Set up command-line arguments
parser = argparse.ArgumentParser()
parser.add_argument('--input', required=True, help='input filepath')
parser.add_argument('--output', required=True, help='output filepath')
parser.add_argument('--targetfreq', type=float, required=True, help='desired key in Hertz')
parser.add_argument('--algorithm', choices=['piano_tuner', 'pymir', 'librosa'], required=True, help='algorithm to use')
parser.add_argument('--samplerate', type=int, required=True, help='sample rate of the audio in Hz')
parser.add_argument('--pitch', type=float, help='pitch of the audio in Hz')
parser.add_argument('--audio', help='audio data as a string')
args = parser.parse_args()

# Load the audio file
if args.audio:
  audio = librosa.core.audio_to_samples(args.audio)
else:
  audio, sample_rate = librosa.load(args.input, sr=args.samplerate)

# Detect the pitch of the audio file
if args.algorithm == 'piano_tuner':
  pitch = piano_tuner.detect_pitch(audio, sample_rate)
elif args.algorithm == 'pymir':
  pitch = pymir.pitch(audio, sample_rate)
elif args.algorithm == 'librosa':
  pitch = librosa.pitch_tuning(audio, sample_rate)
elif args.pitch:
  pitch = args.pitch
else:
  raise ValueError('No pitch detected or provided')

# Convert the pitch to the desired key
if args.algorithm == 'piano_tuner':
  audio = piano_tuner.convert_pitch(audio, sample_rate, pitch, args.targetfreq)
elif args.algorithm in ['pymir', 'librosa']:
  audio = librosa.pitch_shift(audio, sample_rate, pitch, args.targetfreq)

# Save the modified audio file
if args.audio:
  output_audio = lib


In [ ]:
#@markdown python master_script.py --input path/to/input/directory --output path/to/output/directory --minfreq 20 --maxfreq 1000 --length 10000 --sampler resample --samplerate 44100 --pitch_algorithm scipy --shift 5




import argparse
import os
import subprocess

# Parse the arguments from the command line
parser = argparse.ArgumentParser()
parser.add_argument('--input', required=True, dest='input_dir', help='path to the input directory')
parser.add_argument('--output', required=True, dest='output_dir', help='path to the output directory')
parser.add_argument('--minfreq', type=float, help='The lower bound of the fundamental frequency range (in Hz) that the script should consider when processing the audio files')
parser.add_argument('--maxfreq', type=float, help='The upper bound of the fundamental frequency range (in Hz) that the script should consider when processing the audio files')
parser.add_argument('--length', type=int, help='Target length of output audio files in milliseconds')
parser.add_argument('--sampler', choices=['resample', 'resample_poly', 'resample_polyphonic'], help='Librosa resampling function to use')
parser.add_argument('--samplerate', type=int, help='Sample rate to resample audio files to')
parser.add_argument('--pitch_algorithm', choices=['pydub', 'librosa', 'pyaudio', 'pitchy', 'pysndfx', 'sox', 'scipy'], help='pitch shifting algorithm to use')
parser.add_argument('--shift', type=int, default=0, help='pitch shift in semitones')
args = parser.parse_args()

# Directory where the files are stored
directory = args.input_dir

# List of scripts to run on each file
scripts = ['find_fundamental.py', 'trim_stretch.py', 'pitchshift.py']

# Iterate through all the files in the directory
for file in os.listdir(directory):
  # Process the file through each script in the list
  for script in scripts:
    # Build the argument list for the script
    script_args = []
    if script == 'find_fundamental.py':
      script_args += ['--input', args.input_dir]
      script_args += ['--output', args.output_dir]
      if args.minfreq:
        script_args += ['--minfreq', str(args.minfreq)]
      if args.maxfreq:
        script_args += ['--maxfreq', str(args.maxfreq)]
    elif script == 'trim_stretch.py':
      if args.length:
        script_args += ['--length', str(args.length)]
      if args.sampler:
        script_args += ['--sampler', args.sampler]
      if args.samplerate:
        script_args += ['--samplerate', str(args.samplerate)]
    elif script == 'pitchshift.py':
      script_args += ['--input', args.input_dir]
      script_args += ['--output', args.output_dir]
      if args.pitch_algorithm:
        script_args += ['--pitch_algorithm', args.pitch_algorithm]
      if args.shift:
        script_args += ['--shift', str(args.shift)]
    # Run the script with the file and the arguments
    subprocess.run(['python', script, file] + script_args)

#@title MASTER 

#@markdown ---
#@markdown ### INPUT FILE PATH
file_path = "" #@param {type:"string"}
#@markdown ---
#@markdown ### OUTPUT FILE PATH
file_path = "" #@param {type:"string"}
#@markdown ---
#@markdown ### SAMPLE TUNING

#@markdown Tune all samples to desired frequency
SAMPLE TUNING------ = True #@param {type:"boolean"}

#@markdown ---
#@markdown ### TRIM AND STRETCH
#@markdown Trim audio files to specified length, if shorter, make a copy and stretch
TRIM AND STRETCH--- = True #@param {type:"boolean"}
ALGORITHM = 'resample_polyphonic' #@param ['resample', 'resample_poly', 'resample_polyphonic']
TRIM TO:(MS) = 125 #@param {type:"slider", min:50, max:1000, step:25}
SAMPLERATE = 48000 #@param {type:"integer"}
#@markdown ---

#@markdown ### PITCH SHIFTER 
#@markdown Pitch Shift files by desired ammount
PITCH SHIFT-------- = True #@param {type:"boolean"}
PITCH = 0 #@param {type:"slider", min:-12, max:12, step:1}
ALGORITHM = 'PYDUB' #@param ["PYDUB", "LIBROSA", "NUMPY", "PYAUDIO", "PITCHY", "PYSNDFX"]
#@markdown ---

#@markdown ### FREQUENCY DETECTOR
#@markdown Detect frequency and add to filename (audiofile_54hz.wav)
#@markdown
#@markdown SET DETECTION BANDWIDTH WITH MIN - MAX
DETECT FREQUENCY--- = True #@param {type:"boolean"}
MIN FREQ = 40 #@param {type:"slider", min:0, max:1000, step:5}
MAX FREQ = 325 #@param {type:"slider", min:0, max:1000, step:5}

python master_script.py --input path/to/input/directory --output path/to/output/directory --minfreq 20 --maxfreq 1000 --length 10000 --sampler resample --samplerate 44100 --pitch_algorithm scipy --shift 5--ckpt-path $CKPT_PATH\
  --name $NAME\